---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant

___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

[Reference Link](https://github.com/umer7/Understanding-and-Predicting-Property-Maintenance-Fines-using-Machine-Learning/blob/master/Assignment%252B4ml.ipynb)

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Get the train data
# Need to use "ISO-8859-1" encoding
train_df = pd.read_csv("train.csv", encoding = "ISO-8859-1", low_memory = False)
#train_df.head()

In [3]:
# Take a look at the train data for "not responsible" compliance
train_compliance_df = train_df[(train_df["compliance"] == 0) | (train_df["compliance"] == 1)]
#train_compliance_df.head()
#train_compliance_df.shape

In [4]:
# Get the test data
test_df = pd.read_csv("test.csv")
#test_df.head()

In [5]:
# Take a look at the test data and test data for detroit
print("Test Data Shape: {}".format(test_df.shape))
print("Test Data Shape For Detroit: {}".format(test_df[test_df["city"] == "DETROIT"].shape))

Test Data Shape: (61001, 27)
Test Data Shape For Detroit: (26358, 27)


In [6]:
# Get the address data
address_df = pd.read_csv("addresses.csv")
#address_df.head()

In [7]:
# Get the latitude and longitude data
latlon_df = pd.read_csv("latlons.csv")
#latlon_df.head()

In [77]:
# Merge the address with latitude and longitude data
location_df = address_df.set_index("address").join(latlon_df.set_index("address"), how = "left")
#location_df.head()

In [80]:
# Merge the train data with the location data
train_location_df = train_df.set_index("ticket_id").join(location_df.set_index("ticket_id"))
#train_location_df.head()

In [10]:
# Take a look at the combined data
print("Train Data Where Late Fee != 10: {}".format(train_location_df[train_location_df["late_fee"] != 10].shape))

Train Data Where Late Fee != 10: (237790, 35)


In [11]:
# Filter out train data for "not responsible" compliance, keep compliant and non-compliant
train_filter_df = train_location_df[(train_location_df["compliance"] == 0) | (train_location_df["compliance"] == 1)]
#train_filter_df.head()

In [12]:
# Take a look at the data count
print("Train Data Filter Shape: {}".format(train_filter_df.shape))
print("Unique Violation Code Count: {}".format(len(train_filter_df["violation_code"].unique())))
print("Unique City Count: {}".format(len(train_filter_df["city"].unique())))
print("Unique State Count: {}".format(len(train_filter_df["state"].unique())))
print("Unique Train Agency Count: {}".format(len(train_filter_df["agency_name"].unique())))
print("Unique Test Agency Count: {}".format(len(test_df["agency_name"].unique())))
print("Unique Disposition Count: {}".format(len(train_filter_df["disposition"].unique())))

Train Data Filter Shape: (159880, 35)
Unique Violation Code Count: 189
Unique City Count: 4093
Unique State Count: 60
Unique Train Agency Count: 5
Unique Test Agency Count: 3
Unique Disposition Count: 4


In [13]:
train_filter_df["ticket_issued_date"].head()

ticket_id
22056    2004-03-16 11:40:00
27586    2004-04-23 12:30:00
22046    2004-05-01 11:50:00
18738    2004-06-14 14:15:00
18735    2004-06-16 12:30:00
Name: ticket_issued_date, dtype: object

In [14]:
# Check if there are empty ticket issue date
print("Empty Ticket Issue Date: {}".format(train_filter_df[train_filter_df["ticket_issued_date"].isnull()]))

Empty Ticket Issue Date: Empty DataFrame
Columns: [agency_name, inspector_name, violator_name, violation_street_number, violation_street_name, violation_zip_code, mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country, ticket_issued_date, hearing_date, violation_code, violation_description, disposition, fine_amount, admin_fee, state_fee, late_fee, discount_amount, clean_up_cost, judgment_amount, payment_amount, balance_due, payment_date, payment_status, collection_status, grafitti_status, compliance_detail, compliance, lat, lon]
Index: []

[0 rows x 35 columns]


In [15]:
# Filter out hearing date that are empty
train_filter_df = train_filter_df[~train_filter_df["hearing_date"].isnull()]
# Check the amount of data
print("Train Data With Hearing Date Count: {}".format(len(train_filter_df)))
print("Test Data With Hearing Date Count: {}".format(len(test_df[test_df["hearing_date"].isnull()])))
print("Test Data Without Ticket Issue Date Count: {}".format(len(test_df[test_df["ticket_issued_date"].isnull()])))

Train Data With Hearing Date Count: 159653
Test Data With Hearing Date Count: 2197
Test Data Without Ticket Issue Date Count: 0


In [16]:
# Train data hearing date
train_filter_df["hearing_date"].head()

ticket_id
22056    2005-03-21 10:30:00
27586    2005-05-06 13:30:00
22046    2005-03-21 10:30:00
18738    2005-02-22 15:00:00
18735    2005-02-22 15:00:00
Name: hearing_date, dtype: object

In [64]:
from datetime import datetime

def time_delta(hearing, ticket_issued):
    """Calculate the time delta between hearing date and ticket issued date
    
    Args:
        hearing (str): The hearing date
        ticket_issued (str): The ticket issued date
        
    Returns:
        A datetime of the difference between hearing date and ticket issued date
    """
    # There's no hearing date
    if not hearing or type(hearing) is not "str": return 73
    
    # Convert string to datetime
    hearing_date = datetime.strptime(hearing, "%Y-%m-%d %H:%M:%S")
    ticket_issued_date = datetime.strptime(ticket_issued, "%Y-%m-%d %H:%M:%S")
    
    # Calculate the delta
    delta = hearing_date - ticket_issued_date
    
    # Return the delta day(s)
    return delta.days

In [18]:
# Test function
datetime_delta = datetime.strptime("2005-02-22 15:00:00", "%Y-%m-%d %H:%M:%S") - datetime.strptime("2004-06-16 12:30:00", "%Y-%m-%d %H:%M:%S")
print("Time Delta: {}".format(datetime_delta.days))

day_delta = time_delta("2005-02-22 15:00:00", "2004-06-16 12:30:00")
print("Time Delta Function: {}".format(day_delta))

Time Delta: 251
Time Delta Function: 251


In [19]:
# Calculate the time delta for hearing date and ticket issued date
train_filter_df["time_delta"] = train_filter_df.apply(lambda row: time_delta(row["hearing_date"], row["ticket_issued_date"]), axis = 1)
#train_filter_df.head()

In [20]:
train_filter_df["time_delta"].mean()

72.64741032113396

In [21]:
# Feature spliting
feature_split = ["agency_name", "state", "disposition"]

In [22]:
# Check the balance due
"balance_due" in train_filter_df

True

In [130]:
import pandas as pd
import numpy as np

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from datetime import datetime

def blight_model():
    
    # Get train and test data
    train_df = pd.read_csv("train.csv", encoding = "ISO-8859-1", low_memory = False)
    test_df = pd.read_csv("test.csv")
    
    # Filter out train data for "not responsible" compliance, keep compliant and non-compliant
    train_filter_df = train_df[(train_df["compliance"] == 0) | (train_df["compliance"] == 1)]
    
    # Get location data
    address_df = pd.read_csv("addresses.csv")
    latlon_df = pd.read_csv("latlons.csv")
    
    # Merge the address with latitude and longitude data
    location_df = address_df.set_index("address").join(latlon_df.set_index("address"), how = "left")
    # Merge the train filter data with the location data
    train_final_df = train_filter_df.set_index("ticket_id").join(location_df.set_index("ticket_id"))
    # Merge the test data with the address data
    test_final_df = test_df.set_index("ticket_id").join(location_df.set_index("ticket_id"))
    
    # Filter the merged train data, remove entry where there is no hearing date
    train_final_df = train_final_df[~train_final_df["hearing_date"].isnull()]
    
    # Calculate the time delta for train and test data
    train_final_df["time_delta"] = train_final_df.apply(lambda r: time_delta(r["hearing_date"], r["ticket_issued_date"]), axis = 1)
    test_final_df["time_delta"] = test_final_df.apply(lambda r: time_delta(r["hearing_date"], r["ticket_issued_date"]), axis = 1)
    
    # Feature spliting
    feature_split = ["agency_name", "state", "disposition"]
    
    # Fill the empty data for train and test
    train_final_df.lat.fillna(method = "pad", inplace = True)
    train_final_df.lon.fillna(method = "pad", inplace = True)
    train_final_df.state.fillna(method = "pad", inplace = True)
    test_final_df.lat.fillna(method = "pad", inplace = True)
    test_final_df.lon.fillna(method = "pad", inplace = True)
    test_final_df.state.fillna(method = "pad", inplace = True)
    
    # Convert to dummy
    train_final_df = pd.get_dummies(train_final_df, columns = feature_split)
    test_final_df = pd.get_dummies(test_final_df, columns = feature_split)
    
    # Column to remove in train data
    train_column_remove = [
        "balance_due",
        "collection_status",
        "compliance_detail",
        "payment_amount",
        "payment_date",
        "payment_status"
    ]
    
    # Column to remove in all data
    all_column_remove = [
        "fine_amount",
        "violator_name",
        "zip_code",
        "country",
        "city",
        "inspector_name",
        "violation_street_number",
        "violation_street_name",
        "violation_zip_code",
        "violation_description",
        "mailing_address_str_number",
        "mailing_address_str_name",
        "non_us_str_code",
        "ticket_issued_date",
        "hearing_date",
        "grafitti_status",
        "violation_code"
    ]
    
    # Remove the column for train and test data
    train_final_df.drop(train_column_remove, axis = 1, inplace = True)
    train_final_df.drop(all_column_remove, axis = 1, inplace = True)
    test_final_df.drop(all_column_remove, axis = 1, inplace = True)
    # Get the train data feature
    train_feature_index = train_final_df.columns.drop("compliance")
    train_feature_set = set(train_feature_index)
    
    # Loop through and remove train feature not in test data
    for feature in set(train_feature_index):
        if feature not in test_final_df:
            train_feature_set.remove(feature)
            
    train_feature_list = list(train_feature_set)
    
    # Data split
    # Train data
    X_train = train_final_df[train_feature_list]
    y_train = train_final_df.compliance
    # Test data
    X_test = test_final_df[train_feature_list]
    
    # Scale the data
    scale = MinMaxScaler()
    X_train_scale = scale.fit_transform(X_train)
    X_test_scale = scale.transform(X_test)
    
    # Fit the classifier
    clf = MLPClassifier(hidden_layer_sizes = [100, 10],
                        alpha = 5,
                        random_state = 0,
                        solver = "lbfgs",
                        verbose = 0)
    clf.fit(X_train_scale, y_train)
    
    # Predict the probability of the test data scaled
    test_prediction = clf.predict_proba(X_test_scale)[:,1]
    
    # Get the original test data
    test_original_df = pd.read_csv("test.csv", encoding = "ISO-8859-1")
    # Add the prediction to the test data
    test_original_df["compliance"] = test_prediction
    # Set the test data index
    test_original_df.set_index("ticket_id", inplace = True)
    
    return test_original_df.compliance

In [131]:
blight_model()

ticket_id
284932    0.053502
285362    0.007321
285361    0.046114
285338    0.078583
285346    0.104561
            ...   
376496    0.011700
376497    0.011700
376499    0.063579
376500    0.063557
369851    0.116734
Name: compliance, Length: 61001, dtype: float64